In [74]:
import glob
import preproc_tabular
import pandas as pd
import numpy as np
import torch


def duplicate_row(img_dir, data, descriptions, references):
    new_ref = list(references)
    for ref in references:
        images = glob.glob(img_dir + ref + ".jpg")
        idx = new_ref.index(ref)
        new_ref.remove(ref)
        new_ref = new_ref[:idx] + images + new_ref[idx:]

        times = len(images)-1
        if times != 0:
            descriptions = descriptions[:idx] + [descriptions[idx]]*times + descriptions[idx:]
            new_data = data.values
            new_data = np.insert(data.values, idx, [new_data[idx]]*times, axis=0)
            new_data = pd.DataFrame(new_data)
            new_data.columns = data.columns
            data = new_data
            
    return new_data, descriptions, new_ref


img_dir =  "D:\\ORS\\Data\\Images\\"
data, ref, des = preproc_tabular.get_tabular('D:\\ORS\\Data\\Mauli_articoli_222_231_232.xlsx')
ref= ['222_969_64273_00_37C_*', '222_969_64273_01_52I_*', '222_969_61434_00_10A_*']
nd, ndd, nr = duplicate_row(img_dir, data[0:3], des[0:3], ref)

7
1


2


3


4


5


6


